In [1]:
import os.path as op
import warnings

import numpy as np
import pandas as pd
from scipy.signal import correlate, hilbert
import scipy.ndimage as nd

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import get_scorer, make_scorer, r2_score, mean_absolute_error, mean_squared_log_error
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import TransformedTargetRegressor

from utilities import reduce_df_mem_usage, print_score

from matplotlib import pyplot as plt
import seaborn as sns

import gc

In [2]:
#sns.set("dark_background")
#sns.set(context="talk")
gc.enable()

sns.set(style="ticks", context="talk")
plt.style.use("dark_background")
%matplotlib inline

rand_seed = 1234
rand_state = np.random.RandomState(rand_seed)

warnings.simplefilter(action='ignore', category=FutureWarning)

PATH = '/home/dhren/Documents/000_flatiron/002_projects/006_1028_kaggle_ASHRAE/data'

FIGSIZE = (28, 20)

In [3]:
def med_nojunk(x):
    bol = pd.isna(x) | (x == 0)
    return np.median(x[~bol])

def junk(x):
    bol = pd.isna(x) | (x == 0)
    return bol

log_mse = make_scorer(mean_squared_log_error)
mae = make_scorer(mean_absolute_error)

In [4]:
%%time

df_tmp = pd.read_csv(op.join(PATH, 'train.csv'))

df_tmp['timestamp'] = pd.to_datetime(df_tmp['timestamp'])
df_tmp['week'] = pd.Series(df_tmp.timestamp).dt.week.values
df_tmp['dayofweek'] = pd.Series(df_tmp.timestamp).dt.dayofweek.values
df_tmp['hourofday'] = pd.Series(df_tmp.timestamp).dt.hour.values

df_train = reduce_df_mem_usage(df_tmp)


del df_tmp
gc.collect()

display(df_train.head())

Mem. usage decreased to 347.03 Mb (67.9% reduction)


,building_id,meter,timestamp,meter_reading,week,dayofweek,hourofday
0,0,0,2016-01-01,0.0,53,4,0
1,1,0,2016-01-01,0.0,53,4,0
2,2,0,2016-01-01,0.0,53,4,0
3,3,0,2016-01-01,0.0,53,4,0
4,4,0,2016-01-01,0.0,53,4,0


CPU times: user 13.5 s, sys: 545 ms, total: 14.1 s
Wall time: 10.1 s


In [5]:
%%time

group = ['building_id', 'meter']
target = 'meter_reading'

grp = df_train.groupby(group)[[target]]
idx = grp.transform(junk)
df_train.loc[idx.values.ravel(), target] = grp.transform(med_nojunk).loc[idx.values.ravel()]

del idx
gc.collect()

CPU times: user 21.3 s, sys: 796 ms, total: 22.1 s
Wall time: 16.6 s


0

In [6]:
%%time

preds_week = reduce_df_mem_usage(df_train.groupby(['building_id', 'meter', 'week'])[[target]].median().rename({'meter_reading': 'pred_week'},axis=1))
preds_dayofweek = reduce_df_mem_usage(df_train.groupby(['building_id', 'meter', 'dayofweek'])[[target]].median().rename({'meter_reading': 'pred_dayofweek'},axis=1))
preds_hourofday = reduce_df_mem_usage(df_train.groupby(['building_id', 'meter', 'hourofday'])[[target]].median().rename({'meter_reading': 'pred_hourofday'},axis=1))

Mem. usage decreased to  0.95 Mb (0.0% reduction)
Mem. usage decreased to  0.14 Mb (0.0% reduction)
Mem. usage decreased to  0.45 Mb (0.0% reduction)
CPU times: user 4.3 s, sys: 536 ms, total: 4.84 s
Wall time: 4.37 s


In [7]:
%%time

df_tmp = df_train.merge(preds_week, on=['building_id', 'meter', 'week'], how='left')
df_tmp = df_tmp.merge(preds_dayofweek, on=['building_id', 'meter', 'dayofweek'], how='left')
df_tmp = df_tmp.merge(preds_hourofday, on=['building_id', 'meter', 'hourofday'], how='left')

df_train = reduce_df_mem_usage(df_tmp)

del df_tmp
gc.collect()

df_train

Mem. usage decreased to 732.62 Mb (0.0% reduction)
CPU times: user 12.6 s, sys: 1.06 s, total: 13.6 s
Wall time: 6.51 s


,building_id,meter,timestamp,meter_reading,week,dayofweek,hourofday,pred_week,pred_dayofweek,pred_hourofday
0,0,0,2016-01-01 00:00:00,240.944000,53,4,0,240.944000,240.944000,240.944000
1,1,0,2016-01-01 00:00:00,131.871002,53,4,0,131.871002,131.871002,131.871002
2,2,0,2016-01-01 00:00:00,20.476801,53,4,0,20.476801,20.476801,18.019600
3,3,0,2016-01-01 00:00:00,368.924011,53,4,0,368.924011,368.924011,368.924011
4,4,0,2016-01-01 00:00:00,1593.780029,53,4,0,1593.780029,1593.780029,1593.780029
...,...,...,...,...,...,...,...,...,...,...
20216095,1444,0,2016-12-31 23:00:00,8.750000,52,5,23,8.775000,6.975000,8.325000
20216096,1445,0,2016-12-31 23:00:00,4.825000,52,5,23,5.500000,4.925000,4.575000
20216097,1446,0,2016-12-31 23:00:00,9.475000,52,5,23,9.475000,9.475000,5.262500
20216098,1447,0,2016-12-31 23:00:00,159.574997,52,5,23,151.899994,185.925003,208.649994


In [8]:
%%time

mods = {}

bids = df_train.building_id.unique()

for bid in bids:

    df_b = df_train[df_train.building_id == bid]
    
    for mid in df_b.meter.unique():
        #print("{} Building : {} Meter : {} {}".format("=" * 15, bid, mid, "=" * 15))
        
        df_0 = df_b[df_b.meter == mid]

        encode = RobustScaler()
        lr = BaggingRegressor(base_estimator = LinearRegression(), n_estimators = 10, random_state = rand_state)
        clipped_lr = TransformedTargetRegressor(regressor=lr, inverse_func = lambda x : np.clip(x, 0, 1e32), check_inverse=True)
        #lr = LinearRegression()
        pipe = make_pipeline(encode, clipped_lr)
        
        scores = cross_validate(pipe, df_0[["pred_week", "pred_dayofweek", "pred_hourofday"]], df_0["meter_reading"], cv = 5, n_jobs = -1, scoring = {'log_mse' : log_mse, 'r2' : get_scorer('r2'), 'mae' : mae}, return_estimator = True, return_train_score = True)
        
        mods[(bid, mid)] = scores.pop('estimator')
        #print_score(scores)
        #break
    #break

CPU times: user 1min 44s, sys: 3.54 s, total: 1min 47s
Wall time: 4min 6s


In [15]:
%%time

df_tmp = pd.read_csv(op.join(PATH, 'test.csv'))

df_tmp['timestamp'] = pd.to_datetime(df_tmp['timestamp'])
df_tmp['week'] = pd.Series(df_tmp.timestamp).dt.week.values
df_tmp['dayofweek'] = pd.Series(df_tmp.timestamp).dt.dayofweek.values
df_tmp['hourofday'] = pd.Series(df_tmp.timestamp).dt.hour.values

df_test = reduce_df_mem_usage(df_tmp)

del df_tmp
gc.collect()

Mem. usage decreased to 715.79 Mb (67.9% reduction)
CPU times: user 25.1 s, sys: 1.19 s, total: 26.3 s
Wall time: 20.8 s


30

In [16]:
%%time

df_tmp = df_test.merge(preds_week, on=['building_id', 'meter', 'week'], how='left')
df_tmp = df_tmp.merge(preds_dayofweek, on=['building_id', 'meter', 'dayofweek'], how='left')
df_tmp = df_tmp.merge(preds_hourofday, on=['building_id', 'meter', 'hourofday'], how='left')

df_test = reduce_df_mem_usage(df_tmp)

del df_tmp
gc.collect()

Mem. usage decreased to 1511.11 Mb (0.0% reduction)
CPU times: user 27.4 s, sys: 1.37 s, total: 28.8 s
Wall time: 13 s


0

In [31]:
%%time

preds = pd.Series(np.zeros(len(df_test), dtype = 'f8'), index = df_test.index)

bids = df_test.building_id.unique()

for i,bid in enumerate(bids):
    print(i ,"of", len(bids))

    df_b = df_test[df_test.building_id == bid]
    
    
    for mid in df_b.meter.unique():
        
        df_0 = df_b[df_b.meter == mid]
        df_0 = df_0.interpolate(modeth = 'nearest').bfill().ffill()
        bol = (df_test.building_id == bid) & (df_test.meter == mid)
        
        est = mods[(bid, mid)]
        
        pred_0 = np.zeros((5, len(df_0)), dtype = 'f8')
        
        for i in range(5):
            pred_0[i] = est[i].predict(df_0[["pred_week", "pred_dayofweek", "pred_hourofday"]])
            
        preds[bol] = np.median(pred_0, axis = 0)

0 of 1449
1 of 1449
2 of 1449
3 of 1449
4 of 1449
5 of 1449
6 of 1449
7 of 1449
8 of 1449
9 of 1449
10 of 1449
11 of 1449
12 of 1449
13 of 1449
14 of 1449
15 of 1449
16 of 1449
17 of 1449
18 of 1449
19 of 1449
20 of 1449
21 of 1449
22 of 1449
23 of 1449
24 of 1449
25 of 1449
26 of 1449
27 of 1449
28 of 1449
29 of 1449
30 of 1449
31 of 1449
32 of 1449
33 of 1449
34 of 1449
35 of 1449
36 of 1449
37 of 1449
38 of 1449
39 of 1449
40 of 1449
41 of 1449
42 of 1449
43 of 1449
44 of 1449
45 of 1449
46 of 1449
47 of 1449
48 of 1449
49 of 1449
50 of 1449
51 of 1449
52 of 1449
53 of 1449
54 of 1449
55 of 1449
56 of 1449
57 of 1449
58 of 1449
59 of 1449
60 of 1449
61 of 1449
62 of 1449
63 of 1449
64 of 1449
65 of 1449
66 of 1449
67 of 1449
68 of 1449
69 of 1449
70 of 1449
71 of 1449
72 of 1449
73 of 1449
74 of 1449
75 of 1449
76 of 1449
77 of 1449
78 of 1449
79 of 1449
80 of 1449
81 of 1449
82 of 1449
83 of 1449
84 of 1449
85 of 1449
86 of 1449
87 of 1449
88 of 1449
89 of 1449
90 of 1449
91 of 144

KeyboardInterrupt: 

In [30]:
preds

0            211.818689
1             96.046736
2              0.835906
3            191.134913
4           1200.365478
               ...     
41697595       0.000000
41697596       0.000000
41697597       0.000000
41697598       0.000000
41697599       0.000000
Length: 41697600, dtype: float64

In [23]:
df_test['meter_reading'] = preds
df_test[['row_id', 'meter_reading']].to_csv(op.join(PATH, 'submission_1028_linear_targetPeriods_median_forloop.csv'), index = False)

,row_id,building_id,meter,timestamp,week,dayofweek,hourofday,pred_week,pred_dayofweek,pred_hourofday
64,64,53,0,2017-01-01 00:00:00,52,6,0,NaN,725.697998,725.697998
193,193,53,0,2017-01-01 01:00:00,52,6,1,NaN,725.697998,725.697998
322,322,53,0,2017-01-01 02:00:00,52,6,2,NaN,725.697998,725.697998
451,451,53,0,2017-01-01 03:00:00,52,6,3,NaN,725.697998,725.697998
580,580,53,0,2017-01-01 04:00:00,52,6,4,NaN,725.697998,725.697998
...,...,...,...,...,...,...,...,...,...,...
2259494,2259494,53,0,2018-12-31 19:00:00,1,0,19,725.697998,725.697998,725.697998
2259623,2259623,53,0,2018-12-31 20:00:00,1,0,20,725.697998,725.697998,725.697998
2259752,2259752,53,0,2018-12-31 21:00:00,1,0,21,725.697998,725.697998,725.697998
2259881,2259881,53,0,2018-12-31 22:00:00,1,0,22,725.697998,725.697998,725.697998


In [28]:
df_0.pred_week.interpolate(method = 'nearest').bfill().ffill()

64         725.697998
193        725.697998
322        725.697998
451        725.697998
580        725.697998
              ...    
2259494    725.697998
2259623    725.697998
2259752    725.697998
2259881    725.697998
2260010    725.697998
Name: pred_week, Length: 17520, dtype: float32